# Main thesis regressions

Be aware that this will not run unless you have the data stored in the right place. If you are interested please contact the author.

In [1]:
from collections import OrderedDict
from pathlib import Path
from pprint import pprint
import warnings

import linearmodels
import numpy as np
import pandas as pd
import plotly_express as px
import statsmodels.api as sm
from scipy.stats import anderson_ksamp
from tqdm.notebook import tqdm

from load_daily_data import load_frag_data, load_market_quality_statistics, load_copustat

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Load data 

## Fragmentation data

In [2]:
frag = load_frag_data()

# filter
print(frag.shape)
print("First date: \t", frag.index.min())
print("Last date: \t", frag.index.max())

frag.set_index("isin", append=True, inplace=True)

(81612, 9)
First date: 	 2018-07-02 00:00:00
Last date: 	 2019-12-30 00:00:00


## Compustat data

In [3]:
compustat = load_copustat()

## Market quality data

In [4]:
filename = "20200129_09-02-47_liquidity_stats.csv"
filepath = Path(f"../statistics/daily_liquidity/{filename}")
assert filepath.is_file()

In [5]:
# load stats
daily_stats = load_market_quality_statistics(filepath=filepath)

# append "isin" to index
daily_stats.set_index("isin", append=True, inplace=True)

print(daily_stats.shape)
print("First date: \t", daily_stats.index.get_level_values("date").min())
print("Last date: \t", daily_stats.index.get_level_values("date").max())

Initial number of stocks 246

Filter to not include delisted stocks
Num remaining stocks 232

Exclude all stocks that had an IPO later than January 1st 2019
Num remaining stocks 221

(75292, 62)
First date: 	 2018-07-02 00:00:00
Last date: 	 2019-12-30 00:00:00


In [6]:
daily_stats.rename(columns={"num_transactions": "num_orders_aggr"}, inplace=True)
daily_stats.rename(columns={"num_orders_total": "num_orders_passive"}, inplace=True)

In [7]:
daily_stats["quoted_rel_spread_bps_time_weighted"] *= 100
daily_stats["eff_rel_spread_bps_weighted"] *= 100

## Combine the three dataframes into one

In [8]:
# combine
stats = daily_stats.join(frag, how="left", lsuffix="_IMI", sort=False)
stats = stats.join(compustat, how="left", rsuffix="_compu", sort=False)

# first level of index needs to be entity variable
stats = stats.reset_index("date").set_index("date", append=True)

print("First date: \t", stats.index.get_level_values("date").min())
print("Last date: \t", stats.index.get_level_values("date").max())
print(stats.shape)

First date: 	 2018-07-02 00:00:00
Last date: 	 2019-12-30 00:00:00
(75292, 74)


# Create quartiles

### By turnover

In [9]:
# condition = stats.index.get_level_values("date") < pd.Timestamp("2019-07-01")
turnover_stats = stats["turnover"].reset_index("isin").groupby("isin").median()

lower_quartile = turnover_stats["turnover"].quantile(0.25)
median = turnover_stats["turnover"].median()
upper_quartile = turnover_stats["turnover"].quantile(0.75)

conditions = {"3 bottom turnover": turnover_stats["turnover"] < lower_quartile,
              "2 low turnover": (lower_quartile <= turnover_stats["turnover"]) & (turnover_stats["turnover"] < median),
              "1 high turnover": (median <= turnover_stats["turnover"]) & (turnover_stats["turnover"] < upper_quartile),
              "0 top turnover": upper_quartile <= turnover_stats["turnover"]
             }

stats.reset_index("date", inplace=True)

for quartile, condition in conditions.items():
    isins = turnover_stats[condition].index
    stats.loc[isins, "turnover_category"] = quartile 
    
stats.set_index("date", append=True, inplace=True)

In [10]:
num_stocks = stats["turnover_category"].reset_index().groupby("turnover_category")["isin"].nunique()
print(f"Total number of stocks {num_stocks.sum()}")
num_stocks

Total number of stocks 221


turnover_category
0 top turnover       56
1 high turnover      55
2 low turnover       55
3 bottom turnover    55
Name: isin, dtype: int64

### Excluding low turnover stocks?  

In [11]:
# exclude bottom turnover from sample?
stats = stats[~stats["turnover_category"].isin(["3 bottom turnover", "2 low turnover"])]

In [12]:
num_stocks = stats["turnover_category"].reset_index().groupby("turnover_category")["isin"].nunique()
print(f"Total number of stocks {num_stocks.sum()}")
num_stocks

Total number of stocks 111


turnover_category
0 top turnover     56
1 high turnover    55
Name: isin, dtype: int64

In [13]:
relevant_isins = stats.index.get_level_values("isin").unique()
relevant_isins = relevant_isins.to_frame().reset_index(drop=True)
# # Export isins to csv?
# relevant_isins.to_csv("relevant_isins.csv", index=False)

### Market share quartiles

In [14]:
frag_measure = "market_share"  # "non_fragmentation_index" 

In [15]:
frag_per_isin = stats.groupby(["after_nonequivalence", "isin"])[frag_measure].quantile(0.5)
frag_per_isin = frag_per_isin.unstack("after_nonequivalence")
frag_per_isin[frag_measure] = frag_per_isin[True] - frag_per_isin[False]
frag_per_isin.drop(columns=[False, True], inplace=True)

In [16]:
condition = stats.index.get_level_values("date") < pd.Timestamp("2019-07-01")
frag_per_isin = stats.loc[condition, [frag_measure]].reset_index("isin")
frag_per_isin = frag_per_isin.groupby(["isin"]).quantile(0.50)

In [17]:
# # Option 1: simple 
# # a stock is not fragmented, if on more than 50% of all trading days, there was no trading on other venues (see cell above)
# nonfragmentation = frag_per_isin[frag_measure] == 1
# frag_per_isin.loc[nonfragmentation, "fragmentation"] = "not fragmented"
# frag_per_isin.loc[~nonfragmentation, "fragmentation"] = "fragmented"

# Option 2: by quartiles
lower_quartile = frag_per_isin[frag_measure].quantile(0.25)
median = frag_per_isin[frag_measure].median()
upper_quartile = frag_per_isin[frag_measure].quantile(0.75)
conditions = {
    "Q1": frag_per_isin[frag_measure] < lower_quartile,
    "Q2": (lower_quartile <= frag_per_isin[frag_measure]) & (frag_per_isin[frag_measure] < median),
    "Q3": (median <= frag_per_isin[frag_measure]) & (frag_per_isin[frag_measure] < upper_quartile),
    "Q4": upper_quartile <= frag_per_isin[frag_measure],
}
for fragmentation, condition in conditions.items():
    frag_per_isin.loc[condition, "fragmentation"] = fragmentation 

In [18]:
frag_per_isin["fragmentation"].value_counts()

# left join to stats
stats = stats.join(frag_per_isin["fragmentation"], on="isin")

In [19]:
# showing those isin's that did not have 375 observations
num_dates = stats.reset_index().groupby(["fragmentation", "isin"])["date"].nunique()
num_dates[num_dates != 375]

fragmentation  isin        
Q3             CH0002168083    373
               CH0435377954    312
Q4             CH0006227612    374
Name: date, dtype: int64

In [20]:
condition = stats.index.get_level_values("date") < pd.Timestamp("2019-07-01")
num_stocks = stats.reset_index().groupby(["fragmentation"])[["isin"]].nunique()  # .describe()
print(f"Total number of stocks {num_stocks['isin'].sum()}")
num_stocks

Total number of stocks 111


,isin
fragmentation,
Q1,28
Q2,27
Q3,28
Q4,28


In [21]:
# remember: groups can change over time, that's why there are more stocks than total above
stats.reset_index().groupby(["group", "fragmentation"])[["isin"]].nunique()

isin
group fragmentation      
ABck  Q1               14
      Q2               16
      Q3               26
      Q4               28
ACoK  Q1               17
      Q2               13
      Q3                2

In [22]:
stats.reset_index().groupby(["fragmentation", "turnover_category", "group"])[["isin"]].nunique()

isin
fragmentation turnover_category group      
Q1            0 top turnover    ABck      9
                                ACoK     16
              1 high turnover   ABck      5
                                ACoK      1
Q2            0 top turnover    ABck     10
                                ACoK     13
              1 high turnover   ABck      6
Q3            0 top turnover    ABck      7
                                ACoK      2
              1 high turnover   ABck     19
Q4            0 top turnover    ABck      3
              1 high turnover   ABck     25

In [23]:
stats[condition].reset_index().groupby(["fragmentation"])[[frag_measure]].describe()

market_share                                                    \
                     count      mean       std       min       25%       50%   
fragmentation                                                                  
Q1                  6944.0  0.642395  0.077988  0.203277  0.595279  0.645133   
Q2                  6696.0  0.698525  0.075515  0.071510  0.656506  0.704695   
Q3                  6881.0  0.755713  0.090683  0.058995  0.708542  0.764919   
Q4                  6944.0  0.834730  0.093456  0.130761  0.788804  0.847685   

                                   
                    75%       max  
fragmentation                      
Q1             0.691984  0.936183  
Q2             0.748088  0.928688  
Q3             0.815385  1.000000  
Q4             0.899242  1.000000

In [24]:
stats.reset_index().groupby(["after_nonequivalence"])[["isin"]].describe()

isin                          
                      count unique           top freq
after_nonequivalence                                 
False                 27465    111  CH0011029946  248
True                  14094    111  CH0011029946  127

## Market Cap variable

In [25]:
stats["market_cap"] = stats["shares_outstanding"] * stats["price_close"]
market_cap_average_log = np.log(stats.groupby("isin")["market_cap"].mean())
market_cap_average_log.name = "market_cap_average_log"
stats = stats.join(market_cap_average_log)

In [26]:
(stats.reset_index().groupby(["fragmentation"])[["market_cap_average_log"]].describe()).round(2)

market_cap_average_log                                           \
                               count   mean   std    min    25%    50%    75%   
fragmentation                                                                   
Q1                           10500.0  23.18  1.46  20.51  22.02  23.18  23.94   
Q2                           10125.0  22.56  1.13  20.01  21.94  22.62  23.41   
Q3                           10435.0  21.53  0.93  19.89  20.68  21.23  22.20   
Q4                            9750.0  21.12  1.03  19.76  20.38  21.00  21.43   

                      
                 max  
fragmentation         
Q1             26.37  
Q2             24.64  
Q3             23.40  
Q4             24.47

## Fragmentation table

In [27]:
table = list()
for measure in ("market_share", "lit_frag", "market_cap", "turnover"):
    descriptive = stats.reset_index().groupby(["fragmentation"])[[measure]].describe()
    if measure == "market_cap":
        descriptive /= 1e6
        descriptive = descriptive.applymap("{:.0f}".format)
    elif measure == "turnover":
        descriptive /= 1e6
        descriptive = descriptive.applymap("{:.1f}".format)
    else:
        descriptive = descriptive.applymap("{:.2f}".format)
        
    descriptive = descriptive.loc[:, pd.IndexSlice[: , ["mean", "50%", "std"]]]
    table.append(descriptive)
    
table = pd.concat(table, axis=1)
table.rename(
    columns={
        "market_share": "SIX market share",
        "lit_frag": "LitFrag",
        "market_cap": "Market Cap",
        "turnover": "Turnover",
        "mean": "Mean",
        "std": "StDev",
        "50%": "Median"
    }, 
    inplace=True,
)
table = table.T.reindex(["Mean", "Median", "StDev"], level=1).T

num_stocks = stats.reset_index().groupby("fragmentation")["isin"].nunique()
num_stocks = num_stocks.rename("Num stocks").to_frame()
num_stocks.columns = pd.MultiIndex.from_product([num_stocks.columns, ['']])
table = table.join(num_stocks)

for idx in range(4):
    idx += 1
    table.loc[f"Q{idx}", "Fragmentation"] = f"Quartile {idx}"
table.set_index("Fragmentation", inplace=True)

table = table[["Num stocks", "SIX market share", "LitFrag", "Turnover", "Market Cap"]]

In [28]:
table

Num stocks SIX market share              LitFrag               \
                                     Mean Median StDev    Mean Median StDev   
Fragmentation                                                                 
Quartile 1            28             0.76   0.69  0.18    1.80   1.95  0.64   
Quartile 2            27             0.80   0.75  0.15    1.62   1.71  0.51   
Quartile 3            28             0.84   0.82  0.14    1.46   1.47  0.42   
Quartile 4            28             0.89   0.90  0.11    1.29   1.23  0.31   

              Turnover              Market Cap                
                  Mean Median StDev       Mean Median  StDev  
Fragmentation                                                 
Quartile 1        49.5   20.3  74.5      36396  11721  68795  
Quartile 2        25.4   12.2  32.3      11293   6285  13281  
Quartile 3         4.7    2.3   7.5       3533   1855   3761  
Quartile 4         1.8    1.1   2.3       3466   1302   8100

In [29]:
print(table.to_latex())

\begin{tabular}{lrllllllllllll}
\toprule
{} & Num stocks & \multicolumn{3}{l}{SIX market share} & \multicolumn{3}{l}{LitFrag} & \multicolumn{3}{l}{Turnover} & \multicolumn{3}{l}{Market Cap} \\
{} &             Mean & Median & StDev &    Mean & Median & StDev &     Mean & Median & StDev &       Mean & Median &  StDev \\
Fragmentation &            &                  &        &       &         &        &       &          &        &       &            &        &        \\
\midrule
Quartile 1    &         28 &             0.76 &   0.69 &  0.18 &    1.80 &   1.95 &  0.64 &     49.5 &   20.3 &  74.5 &      36396 &  11721 &  68795 \\
Quartile 2    &         27 &             0.80 &   0.75 &  0.15 &    1.62 &   1.71 &  0.51 &     25.4 &   12.2 &  32.3 &      11293 &   6285 &  13281 \\
Quartile 3    &         28 &             0.84 &   0.82 &  0.14 &    1.46 &   1.47 &  0.42 &      4.7 &    2.3 &   7.5 &       3533 &   1855 &   3761 \\
Quartile 4    &         28 &             0.89 &   0.90 &  0.11

## Time variables & dummies

In [30]:
# stats.loc[stats["fragmentation"].isin(["3_little_fragmented", "4_not_fragmented"]), "frag_dummy"] = 0
# stats["frag_dummy"].fillna(value=1, inplace=True)
# stats["frag_dummy"] = stats["frag_dummy"].astype(int)
# stats.reset_index().groupby(["frag_dummy"])[["isin"]].describe()

In [31]:
# stats[stats["frag_dummy"] == 1].index.get_level_values("isin").unique().to_frame().reset_index(drop=True).to_csv("frag_isins.csv", index=False)

In [32]:
dates = stats.index.get_level_values("date")
stats.loc[7 <= dates.month, "half_year"] = "H2"
stats["half_year"].fillna(value="H1", inplace=True)
stats["semester"] = dates.year.astype("str") + "_" + stats["half_year"]

In [33]:
stats["dummy_2019"] = dates.year == 2019

## Calculate daily returns & Amihud 2002

In [34]:
stats.sort_index(inplace=True)

stats["abs_simple_returns"] = np.abs(stats["price_close"] / stats["price_close"].groupby("isin").shift(1) - 1)
stats["amihud"] = stats["abs_simple_returns"] / stats["turnover"] * 1e9  # _simple_simple

stats[["amihud", "semester", "fragmentation"]].groupby(["fragmentation", "semester"]).mean()

amihud
fragmentation semester           
Q1            2018_H2    1.532578
              2019_H1    1.790793
              2019_H2    1.429387
Q2            2018_H2    2.743781
              2019_H1    3.138557
              2019_H2    2.051228
Q3            2018_H2    7.328610
              2019_H1    7.791080
              2019_H2   12.112391
Q4            2018_H2   11.496303
              2019_H1   13.760732
              2019_H2   13.567925

In [35]:
# plot single measure for a quartile
measure = "eff_rel_spread_bps_weighted"
plot_data = stats.loc[stats["fragmentation"] == "Q4", measure].reset_index().dropna()
# px.scatter(plot_data, x="date", y=measure, color="isin")

In [36]:
isin = "CH0012549785"

# measures = ["price_mean", "price_close", "price_log", "price_reciprocal"]
measures = ["quoted_rel_spread_bps_time_weighted", "eff_rel_spread_bps_weighted", "min_tick_size"]
# measures = ["market_cap", "market_cap_average_log", "price_close", "shares_outstanding"]

plot_data = stats.loc[isin, measures]
plot_data = plot_data.stack().reset_index().rename(columns={"level_1": "measure", 0: "value"})
# px.scatter(plot_data, x="date", y="value", color="measure")

# Panel Regressions

## Define regressions

In [37]:
def run_panel_regression(
        data: pd.DataFrame, 
        measures: list,
        control_variables: list,
        entity_effects: bool,
        time_effects: bool
    ):
    detailed_results = OrderedDict()

    for idx, measure in enumerate(measures):

        if measure.startswith(("time", "depth", "num", "message_counts", "value")) and not measure.endswith("percent"):
            dependent = np.log(data[measure])
#             measure = measure + "_log"
        else:
            dependent = data[measure]
            
        if measure == "amihud":
            control_variables = [var for var in exog_vars if var not in ["log_turnover", "RV_slow"]]
            
        elif measure == "RV_slow" or measure == "VSMI":
            control_variables = [var for var in exog_vars if var not in ["VSMI", "RV_slow"]]
        
        elif measure in exog_vars:
            control_variables = [var for var in exog_vars if var != measure]
        
        else:
            control_variables = exog_vars
        
        exogenous = sm.add_constant(data[control_variables])

        model = linearmodels.PanelOLS(dependent=dependent,
                                      exog=exogenous,
                                      entity_effects=entity_effects,
                                      time_effects=time_effects,
                                     )
        try:
            result = model.fit(cov_type='clustered',
                               cluster_entity=True,
                               cluster_time=True,
                              )
        except Exception as exception:
            print(measure)
            print(exception)
            continue

        # store the result
        detailed_results[measure] = result
        
    return detailed_results
    

def deep_dive_coef(detailed_results, variable: str):

    coef_results = pd.DataFrame(columns=["param", "lower", "upper", "tstat", "pvalue"])  # , "lower", "upper"
        
    for measure, result in detailed_results.items():
        param = result.params[variable]
        lower, upper = result.conf_int().loc[variable]
        tstat = result.tstats[variable]
        pvalue = result.pvalues[variable]
        coef_results.loc[measure] = (param, lower, upper, tstat, pvalue)  # , lower, upper
    
    return coef_results



In [38]:
def run_ols(data, measures, exog_vars):

    detailed_results = OrderedDict()
    
    for idx, measure in enumerate(measures):
        
        if measure == "amihud":
            control_variables = [var for var in exog_vars if var not in ["log_turnover", "RV_slow"]]
            
        elif measure == "RV_slow" or measure == "VSMI":
            control_variables = [var for var in exog_vars if var not in ["VSMI", "RV_slow"]]
        
        elif measure in exog_vars:
            control_variables = [var for var in exog_vars if var != measure]
        
        else:
            control_variables = exog_vars
        
        exog = sm.add_constant(data[control_variables])

        if measure.startswith(("time", "depth", "num", "message_counts", "value")) and not measure.endswith("percent"):
            endog = np.log(data[measure])
        else:
            endog = data[measure]

        model = linearmodels.PooledOLS(endog, exog)
        result = model.fit(
            cov_type='clustered',
            cluster_entity=True,
            cluster_time=True,
        )

        # store the result
        detailed_results[measure] = result
        
    return detailed_results

In [39]:
def highlight_lower_than(pvalue):
    if pvalue < 0.01:
        color = "navajowhite"  # "darkgrey"
#             output = "{:.3f} *".format(value)
    elif pvalue < 0.05:
        color = "blanchedalmond"  # "silver"
    elif pvalue < 0.1:
        color = "cornsilk"  # "gainsboro"
    else:
        color = None
    return f"background-color: {color}"

def highlight_significance(data, pvalues):
    background_colors = pvalues.applymap(highlight_lower_than)
    return background_colors

def font_color(value):
    color = 'red' if value < 0 else 'black'
    return f"color: {color}"

def display_results(combined_results):
    
    params = combined_results["param"]
    pvalues = combined_results["pvalue"]
    
    styled = params.round(3).style.applymap(font_color).apply(highlight_significance, pvalues=pvalues, axis=None)

    return styled

In [40]:
def convert_to_significance(pvalue):
    if pvalue < 0.01:
        return "***"
    elif pvalue < 0.05:
        return "**"
    elif pvalue < 0.05:
        return "*"
    else:
        return ""
    
def format_pvalues(series):
    return series.apply(lambda val: val.apply(convert_to_significance))

def format_stars(table, precision=3):
    
    lower = table[["lower"]].round(precision).astype(str)
    lower.columns = lower.columns.droplevel()
    upper = table[["upper"]].round(precision).astype(str)
    upper.columns = upper.columns.droplevel()
    confidence = "[" + lower + ",  " + upper + "]"
    confidence.columns = pd.MultiIndex.from_product([['conf'], confidence.columns])
    
    format_num = "{:." + f"{precision}" + "f}"
    params = table["param"].applymap(lambda num: format_num.format(num))
    pvalues = table["pvalue"]
    tstats = table[["tstat"]].applymap(lambda num: "(" + format_num.format(num) + ")")
    
    params = pvalues.applymap(convert_to_significance) + params
    params.columns = pd.MultiIndex.from_product([['coef'], params.columns])
    
    formatted = pd.concat([params, tstats, confidence])
    formatted.columns.rename("coef_type", level=0, inplace=True)
    formatted = formatted.stack("coef_type")
    formatted.columns.rename("frag_quartile", inplace=True)

    formatted = formatted.reindex(sorted(formatted.columns), axis=1)
    formatted.sort_values(by=["measure", "coef_type"], ascending=True, inplace=True)
    
    return formatted 

In [41]:
liquidity_measures = [
    'quoted_rel_spread_bps_time_weighted',
    'eff_rel_spread_bps_weighted',
    'depth_time_weighted_average',
]
amihud_turnover_measures = ["log_turnover", "RV_slow", "amihud"]

counts_measures = measures = [
    'AT_proxy',
    'num_orders_aggr',
    'num_orders_passive',
    'num_orders_deleted',
    'num_orders_filled',
    'value_entered_mean',
    'value_entered_median',
    'value_entered_total',
    'value_filled_total',
]


all_measures = liquidity_measures + amihud_turnover_measures + counts_measures
measures = all_measures

control_vars = [
#     "RV_slow",
    "VSMI",  # Riordan & Storkenmaier 2012 JFM, p.427, quotes Hendershott & Moulton 2011 JFM, p.583
    "min_tick_size",
    "price_log",
]


explaining_variable = "after_nonequivalence"  # "dummy_2019"

exog_vars = [explaining_variable] + control_vars
exog_vars

['after_nonequivalence', 'VSMI', 'min_tick_size', 'price_log']

## Run the regression

In [42]:
detailed_results = dict()
coef_results = dict()

conditions = {
    "": pd.Series(True, index=stats.index),  # all_
#     "2019_only_": stats.index.get_level_values("date").year == 2019,
#     "H2_only_": stats["half_year"] == "H2",
#     "before_": stats.index.get_level_values("date") < pd.Timestamp("2019-07-01")
}

for condition_name, condition in conditions.items():
    
    subset = stats[condition]
    
#     # Full sample
#     regression_name = f"{condition_name}Full sample"
#     detailed_result = run_panel_regression(subset, measures, exog_vars, entity_effects=True, time_effects=False)
#     detailed_results[regression_name] = detailed_result
#     coef_result = deep_dive_coef(detailed_result, explaining_variable)
#     coef_results[regression_name] = coef_result
    
    # Per fragmentation quartile
    for frag_dummy, data in tqdm(subset.groupby("fragmentation")):

        regression_name = f"{condition_name}{frag_dummy}"
        detailed_result = run_panel_regression(data, measures, exog_vars, entity_effects=True, time_effects=False)
        detailed_results[regression_name] = detailed_result
        coef_result = deep_dive_coef(detailed_result, explaining_variable)
        coef_results[regression_name] = coef_result

### Create the tables

In [43]:
combined = pd.concat(coef_results)
combined.index.set_names(["fragmentation", "measure"], inplace=True)
combined = combined.unstack("fragmentation")
combined.columns.set_names(["coef_type", "fragmentation"], inplace=True)
combined = combined.reindex(combined.columns.sortlevel(level="fragmentation")[0], axis=1)

In [44]:
# Define here which variables we'd like to see
subset = liquidity_measures + amihud_turnover_measures   # counts_measures / liquidity_measures / amihud_turnover_measures

subset = combined.loc[subset].copy()
export_this = format_stars(subset, precision=2)
export_this.reset_index("coef_type", inplace=True)

In [45]:
export_this["coef_type"] = export_this["coef_type"].astype("category")
export_this["coef_type"] = export_this["coef_type"].cat.reorder_categories(["coef", "tstat", "conf"], ordered=True)
export_this = export_this.sort_values(["measure", "coef_type"]).drop(columns="coef_type")

In [46]:
export_this.rename(
    index={
        "quoted_rel_spread_bps_time_weighted": "QSpread",
        "eff_rel_spread_bps_weighted": "ESpread",
        "depth_time_weighted_average": "lnDepth",
        "AT_proxy": "AT_proxy",
        "num_orders_aggr":"Num aggressive Orders",
        "num_orders_deleted": "Num deleted Orders",
        "num_orders_filled": "Num filled Orders",
        "num_orders_passive": "Num passive Orders",
        "value_entered_total": "Log Volume Entered",
        "value_filled_total": "Log Volume Filled",
    },
    columns={col: "Quartile " + col[-1] for col in export_this.columns},
    inplace=True,
)

In [47]:
export_this

frag_quartile,Quartile 1,Quartile 2,Quartile 3,Quartile 4
measure,,,,
RV_slow,***-0.51,-0.26,-0.40,-0.17
RV_slow,(-2.69),(-1.34),(-1.93),(-0.96)
RV_slow,"[-0.89, -0.14]","[-0.63, 0.12]","[-0.81, 0.01]","[-0.52, 0.18]"
amihud,**-0.18,**-0.75,4.99,0.55
amihud,(-2.30),(-2.20),(0.94),(0.54)
amihud,"[-0.33, -0.03]","[-1.42, -0.08]","[-5.45, 15.43]","[-1.43, 2.52]"
lnDepth,-0.00,0.10,0.10,-0.02
lnDepth,(-0.05),(1.83),(1.91),(-0.59)
lnDepth,"[-0.11, 0.11]","[-0.01, 0.21]","[-0.0, 0.2]","[-0.07, 0.04]"


In [48]:
print(export_this.to_latex())

\begin{tabular}{lllll}
\toprule
frag\_quartile &       Quartile 1 &       Quartile 2 &       Quartile 3 &      Quartile 4 \\
measure      &                  &                  &                  &                 \\
\midrule
RV\_slow      &         ***-0.51 &            -0.26 &            -0.40 &           -0.17 \\
RV\_slow      &          (-2.69) &          (-1.34) &          (-1.93) &         (-0.96) \\
RV\_slow      &  [-0.89,  -0.14] &   [-0.63,  0.12] &   [-0.81,  0.01] &  [-0.52,  0.18] \\
amihud       &          **-0.18 &          **-0.75 &             4.99 &            0.55 \\
amihud       &          (-2.30) &          (-2.20) &           (0.94) &          (0.54) \\
amihud       &  [-0.33,  -0.03] &  [-1.42,  -0.08] &  [-5.45,  15.43] &  [-1.43,  2.52] \\
lnDepth      &            -0.00 &             0.10 &             0.10 &           -0.02 \\
lnDepth      &          (-0.05) &           (1.83) &           (1.91) &         (-0.59) \\
lnDepth      &   [-0.11,  0.11] &   [-0.01, 

In [49]:
display_results(combined)

fragmentation,Q1,Q2,Q3,Q4
measure,,,,
quoted_rel_spread_bps_time_weighted,-0.298,0.235,3.994,1.079
eff_rel_spread_bps_weighted,-0.172,0.083,0.79,0.079
depth_time_weighted_average,-0.003,0.101,0.101,-0.016
log_turnover,0.204,0.218,-0.012,0.013
RV_slow,-0.512,-0.256,-0.403,-0.169
amihud,-0.176,-0.749,4.986,0.545
AT_proxy,-1.994,-1.529,0.016,0.174
num_orders_aggr,0.228,0.197,0.07,0.12
num_orders_passive,-0.119,-0.087,0.004,0.138


In [50]:
measure = measures[0]
pprint(measures)
print(f"\nSelected: {measure}")
samples = combined.columns.get_level_values("fragmentation").unique().tolist()
regr_table = linearmodels.panel.compare([detailed_results.get(sample).get(measure) for sample in samples], precision="pvalues")
regr_table

['quoted_rel_spread_bps_time_weighted',
 'eff_rel_spread_bps_weighted',
 'depth_time_weighted_average',
 'log_turnover',
 'RV_slow',
 'amihud',
 'AT_proxy',
 'num_orders_aggr',
 'num_orders_passive',
 'num_orders_deleted',
 'num_orders_filled',
 'value_entered_mean',
 'value_entered_median',
 'value_entered_total',
 'value_filled_total']

Selected: quoted_rel_spread_bps_time_weighted


,Model 0,Model 1,Model 2,Model 3
Dep. Variable,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted
Estimator,PanelOLS,PanelOLS,PanelOLS,PanelOLS
No. Observations,10500,10125,10435,10499
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.0788,0.1698,0.0301,0.0742
R-Squared (Within),0.0788,0.1698,0.0301,0.0742
R-Squared (Between),-0.3657,-0.4255,-6.6548,-7.6863
R-Squared (Overall),-0.2936,-0.3601,-1.6075,-4.4011
F-statistic,298.60,516.17,80.824,209.76
P-value (F-stat),0.0000,0.0000,0.0000,0.0000


# OLS with stock-level controls
Analoguous to Riordan & Storkenmeier 2012, Hendershott & Moulton 2011

Gives similar results as above.

In [51]:
if "market_cap_average_log" not in control_vars:
    control_vars += ["market_cap_average_log"]

exog_vars = [explaining_variable] + control_vars

exog_vars

['after_nonequivalence',
 'VSMI',
 'min_tick_size',
 'price_log',
 'market_cap_average_log']

In [52]:
detailed_results = dict()
coef_results = dict()

conditions = {
    "": pd.Series(True, index=stats.index),  # all_
#     "2019_only_": stats.index.get_level_values("date").year == 2019,
#     "H2_only_": stats["half_year"] == "H2",
#     "before": stats.index.get_level_values("date") < pd.Timestamp("2019-07-01")
}

for condition_name, condition in tqdm(conditions.items()):
    
    subset = stats[condition]
    
#     # Full sample
#     regression_name = f"{condition_name}Full sample"
#     detailed_result = run_panel_regression(subset, measures, exog_vars, entity_effects=True, time_effects=False)
#     detailed_results[regression_name] = detailed_result
#     coef_result = deep_dive_coef(detailed_result, explaining_variable[0])
#     coef_results[regression_name] = coef_result
    
    # Per fragmentation quartile
    for frag_dummy, data in subset.groupby("fragmentation"):

        regression_name = f"{condition_name}{frag_dummy}"
        detailed_result = run_ols(data, measures, exog_vars)
        detailed_results[regression_name] = detailed_result
        coef_result = deep_dive_coef(detailed_result, explaining_variable)
        coef_results[regression_name] = coef_result

In [53]:
combined = pd.concat(coef_results)
combined.index.set_names(["fragmentation", "measure"], inplace=True)
combined = combined.unstack("fragmentation")
combined.columns.set_names(["coef_type", "fragmentation"], inplace=True)
combined = combined.reindex(combined.columns.sortlevel(level="fragmentation")[0], axis=1)

In [54]:
export_this = format_stars(combined, precision=3)
# print(export_this.to_latex(sparsify=True))
export_this

frag_quartile                                                 Q1  \
measure                             coef_type                      
AT_proxy                            coef               ***-1.967   
                                    conf        [-2.764,  -1.17]   
                                    tstat               (-4.837)   
RV_slow                             coef                **-0.539   
                                    conf        [-1.038,  -0.04]   
                                    tstat               (-2.119)   
amihud                              coef                **-0.147   
                                    conf       [-0.289,  -0.004]   
                                    tstat               (-2.012)   
depth_time_weighted_average         coef                  -0.012   
                                    conf        [-0.119,  0.094]   
                                    tstat               (-0.226)   
eff_rel_spread_bps_weighted         coef                  -0.194   
                                    conf        [-0.443,  0.054]   
                                    tstat               (-1.531)   
log_turnover                        coef                ***0.191   
                                    conf         [0.107,  0.276]   
                                    tstat                (4.433)   
num_orders_aggr                     coef                ***0.223   
                                    conf         [0.143,  0.302]   
                                    tstat                (5.490)   
num_orders_deleted                  coef               ***-0.169   
                                    conf       [-0.273,  -0.065]   
                                    tstat               (-3.182)   
num_orders_filled                   coef                ***0.293   
                                    conf         [0.219,  0.367]   
                                    tstat                (7.747)   
num_orders_passive                  coef                **-0.128   
                                    conf       [-0.228,  -0.028]   
                                    tstat               (-2.504)   
quoted_rel_spread_bps_time_weighted coef                  -0.363   
                                    conf        [-0.945,  0.219]   
                                    tstat               (-1.222)   
value_entered_mean                  coef               ***-0.095   
                                    conf       [-0.138,  -0.051]   
                                    tstat               (-4.281)   
value_entered_median                coef               ***-0.214   
                                    conf       [-0.259,  -0.169]   
                                    tstat               (-9.336)   
value_entered_total                 coef               ***-0.276   
                                    conf        [-0.39,  -0.161]   
                                    tstat               (-4.718)   
value_filled_total                  coef                ***0.185   
                                    conf         [0.099,  0.272]   
                                    tstat                (4.195)   

frag_quartile                                                 Q2  \
measure                             coef_type                      
AT_proxy                            coef               ***-1.539   
                                    conf       [-2.083,  -0.994]   
                                    tstat               (-5.543)   
RV_slow                             coef                **-0.347   
                                    conf       [-0.689,  -0.005]   
                                    tstat               (-1.990)   
amihud                              coef                **-0.769   
                                    conf       [-1.471,  -0.068]   
                                    tstat               (-2.149)   
depth_time_weighted_average         coef               

In [55]:
display_results(combined)

fragmentation,Q1,Q2,Q3,Q4
measure,,,,
quoted_rel_spread_bps_time_weighted,-0.363,0.159,4.037,2.659
eff_rel_spread_bps_weighted,-0.194,0.057,0.834,0.765
depth_time_weighted_average,-0.012,0.11,0.1,-0.062
log_turnover,0.191,0.229,-0.019,-0.072
RV_slow,-0.539,-0.347,-0.432,-0.05
amihud,-0.147,-0.769,4.82,1.635
AT_proxy,-1.967,-1.539,0.013,0.27
num_orders_aggr,0.223,0.203,0.066,0.076
num_orders_passive,-0.128,-0.076,-0.002,0.093


In [56]:
pprint(measures)
measure = measures[0]
print(f"\nSelected: {measure}")
samples = combined.columns.get_level_values("fragmentation").unique().tolist()
linearmodels.panel.compare([detailed_results.get(sample).get(measure) for sample in samples], precision="pvalues")

['quoted_rel_spread_bps_time_weighted',
 'eff_rel_spread_bps_weighted',
 'depth_time_weighted_average',
 'log_turnover',
 'RV_slow',
 'amihud',
 'AT_proxy',
 'num_orders_aggr',
 'num_orders_passive',
 'num_orders_deleted',
 'num_orders_filled',
 'value_entered_mean',
 'value_entered_median',
 'value_entered_total',
 'value_filled_total']

Selected: quoted_rel_spread_bps_time_weighted


,Model 0,Model 1,Model 2,Model 3
Dep. Variable,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted,quoted_rel_spread_bps_time_weighted
Estimator,PooledOLS,PooledOLS,PooledOLS,PooledOLS
No. Observations,10500,10125,10435,9750
Cov. Est.,Clustered,Clustered,Clustered,Clustered
R-squared,0.6760,0.7327,0.0826,0.2012
R-Squared (Within),0.0492,0.0857,0.0181,0.0371
R-Squared (Between),0.7973,0.8126,0.2755,0.3271
R-Squared (Overall),0.6760,0.7327,0.0826,0.2012
F-statistic,5474.3,5548.6,187.72,490.94
P-value (F-stat),0.0000,0.0000,0.0000,0.0000
